In [ ]:
# import the libraries
import tensorflow as tf
from keras._tf_keras.keras import layers, ops, callbacks, Model
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [ ]:
# define the file path
file_path = "datasets/fuel_optimal_db.csv"

# Load the data using pandas
data = pd.read_csv(file_path)

# drop unused columns
columns_to_drop = ["mima [kg]", "mima2 [kg]", "mima3 [kg]", "mfms_approx"]
data = data.drop(columns=columns_to_drop)

# Preprocess data
y = data["mt [kg]"] / data["ms [kg]"]
X = data.drop("mt [kg]", axis=1)

# split data in train and test -- 20% of the dataset is used for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Multi Layer Perceptron
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(X_train.shape[1],)),
        layers.BatchNormalization(),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_absolute_error"])

In [ ]:
# fit the model
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    verbose=1,
    batch_size=32,
)

In [ ]:
# evaluate and check the history
test_loss = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_test_scaled)
train_mae = history.history["mean_absolute_error"]

In [ ]:

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(train_mae, label='Training MAE')
plt.title('Mean Absolute Error Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()